In [ ]:
### A Pluto.jl notebook ###
# v0.14.7

using Markdown
using InteractiveUtils

In [ ]:
using BenchmarkTools

In [ ]:
using FixArgs

In [ ]:
md"""
Suppose we have a `Vector` of `Vector`s, and we want to concatenate all of the inner `Vector`s into one `Vector`.
"""

In [ ]:
vs = [1:100 for _ = 1:200]; # `Vector` of `<:AbstractVector`,  really.

In [ ]:
md"""
To concatenate two `AbstractVector`s, use `vcat`:
"""

In [ ]:
vcat(1:5, 1:5)

In [ ]:
md"""
Apply binary operation over a sequence using `reduce`:
"""

In [ ]:
reduce(vcat, vs)

In [ ]:
md"""
Let's time it:
"""

In [ ]:
@benchmark reduce(vcat, vs)

In [ ]:
md"""
Now let us do essentially the same computation, but instead of directly using `vcat`, we define a function (an anonymous function) that is just a wrapper around `vcat`.
"""

In [ ]:
@benchmark reduce((_1, _2) -> vcat(_1, _2), vs)

In [ ]:
md"""
It is ~100× slower in this case.
It is not because anonymous functions are slow.
"""

In [ ]:
methods(reduce)

In [ ]:
md"""
Multiple dispatch and each-function-is-a-type allow us to use a special case that allocates the result all at once, with the spelling `reduce(vcat, ...)`. Without these features, one would instead need to make a new name like e.g. `reduce_vcat(...)`.

Personally, the first spelling is better because it combines existing and meaningful names instead of introducing a new ad-hoc name.

Note that `reduce(vcat, ...)` might not even call `vcat`, but that `vcat` is used as a _name_. More so than in other ecosystems, Julia tries to pin down the meaning of function names to enable generic programming.
"""

In [ ]:
md"""
There are multiple converging motivations for the idea in this talk:
* Get extra value from careful function name / meaning pairs
* Generalize `Base.Fix1`/`Base.Fix2`
* Symbolic Computation / Lazy Computation
* structural vs nominal
"""

In [ ]:
f1 = ==(50)

In [ ]:
f2 = x -> x == 50

In [ ]:
md"""
`f1` and `f2` compute the same function. Some might say that they are different names for the name function.

And in this case, names matter!
"""

In [ ]:
findfirst(f1, 1:100)

In [ ]:
findfirst(f2, 1:100)

In [ ]:
@which findfirst(f1, 1:100)

In [ ]:
md"""
```julia
findfirst(p::Union{Fix2{typeof(isequal),T},Fix2{typeof(==),T}}, r::AbstractUnitRange) where {T<:Integer} =
    first(r) <= p.x <= last(r) ? 1+Int(p.x - first(r)) : nothing
```
"""

In [ ]:
md"""
This kind of thing is why I love Julia. Imagine for example a plotting library that supports unevenly spaced axis ticks.
If you write some code in terms of `findfirst`, then it can support unevenly spaced ticks, and still be (runtime) efficient when using evenly spaced ticks.
"""

In [ ]:
md"""
`Fix1`/`Fix2` fix one argument of a two-argument function.

Would it ever be useful to fix all of the arguments of a function?
"""

In [ ]:
md"""
Consier the `/` function.
If you fix its two arguments, that's pretty much `Rational`.
"""

In [ ]:
half = @xquote 1 / 2

In [ ]:
function Base.:*(a::(@xquoteT ::S / ::S), b::(@xquoteT ::S / ::S)) where S
	(n1, d1) = something.(Tuple(a.args))
	(n2, d2) = something.(Tuple(b.args))
	@show n1 d1 n2 d2
	@xquote $(n1 * n2) / $(d1 * d2)
end
	

In [ ]:
half * half

In [ ]:
md"""
Using this instead of `Base.Rational` seems pretty silly, until you consider
* Some users want a "rational" type where the numerator and the denominator are not constrained to be the same type.
* a fixed-point number is one of these rational types where the denominator is "static" (a singleton type such that the numerical value is encoded in the type domain)
"""

In [ ]:
ft.